## Genetic

In [8]:
import random
from PIL import Image, ImageDraw
import numpy as np
import cv2

from copy import deepcopy

In [9]:
def resize(image, max_size):
    new_width = int((max_size/max(image.size[0],image.size[1]))* image.size[0])
    new_height = int((max_size/max(image.size[0],image.size[1]))* image.size[1])
    image = image.resize((new_width,new_height), resample=Image.Resampling.LANCZOS)  
    return image

In [10]:
OFFSET = 10

def generate_point(width, height):
    x = random.randrange(0 - OFFSET, width + OFFSET, 1)
    y = random.randrange(0 - OFFSET, height + OFFSET, 1)
    return (x, y)

class Triangle:
    def __init__(self, img_width, img_height):
        self.points = []
        for i in range(3):
            self.points.append(generate_point(img_width,img_height))
        

        self.color = (
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
        )

        self._img_width = img_width
        self._img_height = img_height

In [11]:
class Chromosome:  
    def __init__(self,img_height, img_width,target_image,num_triangles):
        self.img_height = img_height
        self.img_width = img_width  
        self.background_color = (0,0,0,255)
        self.triangles = []
        self.target_image = target_image
        self.num_trinagles = num_triangles

        for _ in range(num_triangles):
            self.triangles.append(Triangle(img_width, img_height))

    def mutate(self):
        triidx = random.randint(0, self.num_trinagles - 1)
        pidx = random.randint(0, 2)
        self.triangles[triidx].points[pidx] = generate_point(self.img_width, self.img_height)

        self.triangles[triidx].color = (
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
        )
    
    def draw(self) -> Image:
        size = self.target_image.size
        img = Image.new('RGB', size, self.background_color)
        draw = Image.new('RGBA', size)
        pdraw = ImageDraw.Draw(draw)
        for triangle in self.triangles:
            colour = triangle.color
            points = triangle.points
            pdraw.polygon(points, fill=colour, outline=colour)
            img.paste(draw, mask=draw)
        return img
        

    def fitness(self) -> float:
        created_image = np.array(self.draw()) / 255.0
        target_image = np.array(self.target_image) / 255.0
           
        mse = np.mean((created_image - target_image) ** 2)
        return mse

In [12]:
class GeneticAlgorithm():
    def __init__(self,max_width,max_height,target_image, population_size, triangles_number):
        self.population_size = population_size
        self.max_width = max_width
        self.max_height = max_height
        self.population = [Chromosome(max_height,max_width,target_image, triangles_number) for _ in range(population_size)]
        self.target_image = target_image
    
    def calc_fitnesses(self):
        fitnesses = []
        for chromosome in self.population:  
            fitnesses.append(chromosome.fitness())
        return fitnesses
    
    def sort_population(self, fitnesses):
        return [x for _, x in sorted(zip(fitnesses, self.population), key=lambda pair: pair[0])]

    def cross_over(self, parent1 : Chromosome, parent2 : Chromosome):
        child = Chromosome(self.max_height, self.max_width, self.target_image, parent1.num_trinagles)
        cross_over_point = random.randint(0, parent1.num_trinagles)
        child.triangles = parent1.triangles[:cross_over_point] + parent2.triangles[cross_over_point:]
        return deepcopy(child)
    
    def run(self, n_generations):
        for iteration in range(n_generations):
            new_population = []
            fitnesses = self.calc_fitnesses()
        
            self.population = self.sort_population(fitnesses)

            if iteration % 10 == 0:
                fit_arr = np.array(fitnesses)
                print(f"Fitness in Generation {iteration}: mean: {np.mean(fit_arr)}, max: {np.max(fit_arr)} min: {np.min(fit_arr)}")
                if iteration % 100 == 0:
                    best_img = self.population[0].draw()
                    best_img.save(f"./saved_imgs/best_img_{iteration}.png")
                    print(f"saved best of gen {iteration}.")

            TOP = 0.2
            top_cnt = int(TOP * self.population_size)
            new_population.extend(self.population[:top_cnt])

            while len(new_population) < self.population_size:
                parent1 = random.choice(new_population[:top_cnt])
                parent2 = random.choice(new_population[:top_cnt])

                child = self.cross_over(parent1, parent2)
                new_population.append(child)
                
            
            self.population = new_population
            MU_PROB = 0.5
            for chromo in self.population[top_cnt:]:
                if random.random() < MU_PROB:
                    chromo.mutate()

In [13]:
def resize(image,max_size):
    new_width = int((max_size/max(image.size[0],image.size[1]))* image.size[0])
    new_height = int((max_size/max(image.size[0],image.size[1]))* image.size[1])
    image = image.resize((new_width,new_height), resample=Image.Resampling.LANCZOS)  
    return image

In [14]:
target_image_path = "./target_images/sunset.jpeg"
image = Image.open(target_image_path)
image = resize(image, 50)
image.show()

width, height = image.size
population_size = 200
triangles_number = 50
alg = GeneticAlgorithm(width, height, image, population_size, triangles_number)
alg.run(1001)

Fitness in Generation 0: mean: 0.12073652890302487, max: 0.18674511668006474 min: 0.06471900861750919
saved best of gen 0.
Fitness in Generation 10: mean: 0.043090174641454926, max: 0.07277373815043942 min: 0.040400048155526815


[46663:46663:1106/193427.518562:ERROR:CONSOLE(0)] "The source list for Content Security Policy directive 'frame-src' contains a source with an invalid path: '/?ncedge=1&features=-'. The query component, including the '?', will be ignored.", source: about:blank (0)
[46663:46663:1106/193427.518587:ERROR:CONSOLE(0)] "Unrecognized Content-Security-Policy directive 'ncedge'.", source: about:blank (0)
[46663:46663:1106/193427.518604:ERROR:CONSOLE(0)] "Unrecognized Content-Security-Policy directive 'edgepagecontext'.", source: about:blank (0)
[46663:46663:1106/193427.518676:ERROR:CONSOLE(0)] "The source list for Content Security Policy directive 'frame-src' contains a source with an invalid path: '/?ncedge=1&features=-'. The query component, including the '?', will be ignored.", source: about:blank (0)
[46663:46663:1106/193427.518685:ERROR:CONSOLE(0)] "Unrecognized Content-Security-Policy directive 'ncedge'.", source: about:blank (0)
[46663:46663:1106/193427.518699:ERROR:CONSOLE(0)] "Unrecogn

Fitness in Generation 20: mean: 0.037126978062050724, max: 0.0789704441570647 min: 0.035544532600126605
Fitness in Generation 30: mean: 0.03440950262331116, max: 0.06615187787137038 min: 0.03316628336420724
Fitness in Generation 40: mean: 0.03251040333360518, max: 0.05321775385535477 min: 0.03162428747296106
Fitness in Generation 50: mean: 0.030903496308723532, max: 0.060022803972054256 min: 0.028970734643629682
Fitness in Generation 60: mean: 0.028838120971343577, max: 0.06495235165961809 min: 0.02778357663524907
Fitness in Generation 70: mean: 0.027694955180408466, max: 0.039950757090318804 min: 0.027129751960201787
Fitness in Generation 80: mean: 0.027946102532437015, max: 0.053071398335527514 min: 0.026794651629715067
Fitness in Generation 90: mean: 0.02731146791249675, max: 0.0417614732484398 min: 0.026479030986528106
Fitness in Generation 100: mean: 0.02722607008182556, max: 0.05894298929316231 min: 0.026203966617346083
saved best of gen 100.
Fitness in Generation 110: mean: 0.02

### Q1.

One chromosome as a potential solution, has a fixed number of triangles (say n). each triangles is identified with its color (4 parameters) and the position of its vertices (3 parameters). If we assume that we select all 7 parametes as natural numbers, for a singles triangle we would have a total of $m = 256^4 \times (h \times w)^3$ possible ways. So the total number of states would be $ m^n $ which is ridiculously large number.

### Q2.

We could use circles instead of triangles to get more details of the original image since circles can be a better representation of pixels.
Choosing a good crossover strategy can benefit the speed of algorithm significantly. For instance we could use TOP 5% of the population and use them to generate new samples to get a higher accuracy. This can be speed up the algorithm because we are using the best of the population and trying to get some sort of combination as new samples with them. The strategy for mutation is also crucial for the running time of algorithm. For example if we could come up with some idea to mutate the colors of triangles systematically, not just randomly, we would get better results. 

Another suggestion would be to limit the size of the triangles and make them small to get better resolution of the image. Another idea would be to increase the number of triangles randomly. This could slow down the run time of the algorithm on some cases, but over all I think it would generate better answers and potentially generate them faster in some cases.

### Q3.

I think the best option for choosing the next population, would be the one I implemented with some fine tuning of course. We choose the TOP 10% of the population and transfer them directly without mutating them to the next population, and run crossover on them to generate their combination randomly.

Other alternatives would be run mutation on the TOP% of the population (Which is awful) or we could run crossover on the recently generated candidates too to fill the new population.